**Исследование надежности заемщиков**
=

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Обзор данных

In [1]:
#импорт библиотек

import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


# Предобработка данных 

## Удаление пропусков

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [4]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = data.loc[(data['income_type'] == t), 'total_income'].median()

## Обработка аномальных значений 

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.

In [5]:
data['days_employed'] = data['days_employed'].abs()

In [6]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Нужно удалить строки, в которых встречаются такие аномальные значения.

In [8]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [9]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = data.loc[(data['income_type'] == t), 'days_employed'].median()

In [10]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски заполнены.

## Изменение типов данных

In [11]:
data['total_income'] = data['total_income'].astype(int)

## Обработка дубликатов

In [12]:
data['education'] = data['education'].str.lower()

In [13]:
data.duplicated().sum()

71

In [14]:
data = data.drop_duplicates()

## Категоризация данных

На основании диапазонов, указанных ниже, нужно создать в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [15]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [16]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [17]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Нужно создать функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [18]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [19]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

# Исследование гипотез

## Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [20]:
for child in data['children'].unique():
    total_child_data = data.loc[data['children'] == child, 'debt'].count() 
    data.loc[data['children'] == child, 'total_child_data'] = total_child_data 
    debt_child_data = data.loc[data['children'] == child, 'debt'].sum() 
    data.loc[data['children'] == child, 'fraction_debt_child'] = debt_child_data / total_child_data 

data.groupby(['children', 'total_child_data'])['fraction_debt_child'].median()

children  total_child_data
0         14091.0             0.075438
1         4808.0              0.092346
2         2052.0              0.094542
3         330.0               0.081818
4         41.0                0.097561
5         9.0                 0.000000
Name: fraction_debt_child, dtype: float64

**Вывод:** 
Стоит отметить, что количество заемщиков с 3, 4 и 5 детьми очень немногочисленны, поэтому эти категории для выявления зависимости использоваться не будут. Если же посмотреть на категории без детей и с 1 или 2 детьми, то видно, что с увеличением количества детей увеличивается процент кредитов, невозвращенных вовремя.

## Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
for family in data['family_status'].unique():
    total_family_data = data.loc[data['family_status'] == family, 'debt'].count()
    data.loc[data['family_status'] == family, 'total_family_data'] = total_family_data
    debt_family_data = data.loc[data['family_status'] == family, 'debt'].sum()
    data.loc[data['family_status'] == family, 'fraction_debt_family'] = debt_family_data / total_family_data 

data.groupby(['family_status', 'total_family_data'])['fraction_debt_family'].median()

family_status          total_family_data
Не женат / не замужем  2796.0               0.097639
в разводе              1189.0               0.070648
вдовец / вдова         951.0                0.066246
гражданский брак       4134.0               0.093130
женат / замужем        12261.0              0.075606
Name: fraction_debt_family, dtype: float64

**Вывод:** Можно данные категории разделить еще на 2 категории: те кто есть или был в официальном браке (группы женат/замужем, в разводе, вдовец / вдова) и те кто в нем не был ( не женат/не замужем, гражданский брак). Так вот люди 1-ой объединенной категории чаще возвращают кредиты вовремя, чем люди 2-ой объединенной категории.

## Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
for income in data['total_income_category'].unique(): 
    total_income_data = data.loc[data['total_income_category'] == income, 'debt'].count() 
    data.loc[data['total_income_category'] == income, 'total_income_data'] = total_income_data 
    debt_income_data = data.loc[data['total_income_category'] == income, 'debt'].sum() 
    data.loc[data['total_income_category'] == income, 'fraction_debt_income'] = debt_income_data / total_income_data 

data.groupby(['total_income_category', 'total_income_data'])['fraction_debt_income'].median() 

total_income_category  total_income_data
A                      25.0                 0.080000
B                      5014.0               0.070602
C                      15921.0              0.084982
D                      349.0                0.060172
E                      22.0                 0.090909
Name: fraction_debt_income, dtype: float64

**Вывод:** Категории заемщиков с уровнями доходами А, D и Е являются очень не многочисленными, поэтому их нельзя использовать для выявления общей зависимости. 
Из данных оставшихся групп можно сделать вывод, что чем меньше уровень дохода, тем выше процент невозвращенных кредитов.
Возможно стоит изменить подход к категоризациюю доходов, чтобы многочисленными были хотя бы 3-4 группы.

## Как разные цели кредита влияют на его возврат в срок?

In [23]:
for purpose in data['purpose_category'].unique(): 
    total_purpose_data = data.loc[data['purpose_category'] == purpose, 'debt'].count() 
    data.loc[data['purpose_category'] == purpose, 'total_purpose_data'] = total_purpose_data 
    debt_purpose_data = data.loc[data['purpose_category'] == purpose, 'debt'].sum() 
    data.loc[data['purpose_category'] == purpose, 'fraction_debt_purpose'] = debt_purpose_data / total_purpose_data 

data.groupby(['purpose_category', 'total_purpose_data'])['fraction_debt_purpose'].median()

purpose_category          total_purpose_data
операции с автомобилем    4279.0                0.093480
операции с недвижимостью  10751.0               0.072551
получение образования     3988.0                0.092528
проведение свадьбы        2313.0                0.079118
Name: fraction_debt_purpose, dtype: float64

**Вывод:** Чаще всего не возвращают кредит люди, берущие его с целью операций с автомобилями. Реже всего - берущие на операции с недвижимостью.

## Приведите возможные причины появления пропусков в исходных данных

*Ответ:* Не был указан стаж в днях и размер зарплаты при заполнении заявки на кредит - либо забыли, либо человек не хотел афишировать эти данные, либо человек не имеет и никогда не имел официальной работы (а соответственно данных о стаже и зарплате), либо не было нужды указывать эти данные, потому как кредит выдавали и без этих данных.

Возможно возникла проблема при выгрузке данных из системы банка или данные были занесены не в том формате.

## Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных

*Ответ:* Использование среднего значения может быть некорректно для данных, в которых представлены значения, сильно выделяющиеся среди других. Поэтому лучше использовать медианное значение, которое меньше подвержено изменениям при сильно разнящихся значениях.

# Общий вывод

В данной работе были проанализированы данные от банка - сведения о платежеспособности клиентов - с целью выявления зависимости выплаты кредита в срок от уровня дохода, количества детей, семейного положения и цели кредита. 

Анализ полученных данных показал, что:

1. При увеличении количества детей увеличивается вероятность невозврата кредита в срок.
2. Наличие статуса официального брака (в настоящем или прошедшем) увеличивает шансы на своевременный возврат кредита.
3. При снижении уровня дохода увеличивается вероятность невозврата кредита в срок.
4. Чаще всего не возвращают кредит люди, берущие его с целью операций с автомобилями.

